## Quick reflection prototype (08-03-24)

#Installs

In [ ]:
#installs
!pip install openai
!pip install --upgrade openai

#Set User Reserach Goal

In [ ]:

user_research_focus = "Quality of the reflection questions." # @param {type:"string"}


#Main Protoype

In [ ]:
import os
import openai
import json
from IPython.display import display, HTML
import time

# Initialize OpenAI API
client = openai.OpenAI(api_key='sk-x')

# Global variables
reflection_questions = {}
user_research_feedback = ""

# Aesthetics for printing conversations
def pretty_print_conversation(messages, start_index=0):
    formatted_message = ""
    role_to_color = {
        "system": "#4DD8FF",
        "assistant": "#5B61FF",
        "tool": "#FF87AB",
    }
    font_size = "16px"
    font_weight = "500"  # Normal font weight
    system_font_family = "Courier New, monospace"
    writer_font_family = "Arial, sans-serif"
    margin_top = "5px"  # Top margin
    margin_bottom = "5px"  # Bottom margin

    for message in messages[start_index:]:
        if not isinstance(message, dict):
            # Skip non-dictionary messages
            continue

        if message.get("role") == "user":
            continue  # Skip user messages

        color = role_to_color.get(message.get("role"), "black")
        display_role = message.get("role", "").capitalize()
        font_family = writer_font_family if message.get("role") == "assistant" else system_font_family

        if message.get("role") == "tool":
            # Special handling for tool messages
            tool_label = f"<span style='color: {color};'>Tool:</span>"
            tool_message = f" Function ({message['name']}): {message['content']}"
            formatted_message += f"<span style='font-size: {font_size}; font-family: {font_family}; font-weight: {font_weight}; margin-top: {margin_top}; margin-bottom: {margin_bottom}; display: block;'>{tool_label}{tool_message}</span><br>"
        else:
            # Handling for other message types
            formatted_message += f"<span style='font-size: {font_size}; font-family: {font_family}; font-weight: {font_weight}; margin-top: {margin_top}; margin-bottom: {margin_bottom}; display: block;'><span style='color: {color};'>{display_role}</span>: {message['content']}</span><br>"

    display(HTML(formatted_message))



def get_openai_response(conversation, tools=[]):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=conversation,
        tools=tools,
        tool_choice="required",
        temperature=0.5,
        max_tokens=4095
    )
    return completion

def add_reflection_question(question):
    global reflection_questions
    question_number = len(reflection_questions) + 1
    reflection_questions[f"question_{question_number}"] = {"question": question, "answer": ""}
    return json.dumps({"status": "success", "message": f"Added reflection question {question_number}"})

def add_user_research_question(question):
    return json.dumps({"status": "success", "message": f"Added user research question: {question}"})

def save_user_feedback(feedback):
    global user_research_feedback
    user_research_feedback = feedback
    with open("user_research_feedback.txt", "w") as f:
        f.write(feedback)
    return json.dumps({"status": "success", "message": "User feedback saved"})

def interactive_chat():
    system_prompt = """You are an expert AI reflection app. Your goal is to generate three reflection questions following the principles of the reflective practitioner. This is the user input you are generating reflection questions for: {initial_user_input}"""

    conversation = [
        {"role": "system", "content": system_prompt},
    ]

    tools = [
        {
            "type": "function",
            "function": {
                "name": "add_reflection_question",
                "description": "Add a reflection question",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "The reflection question to add"
                        }
                    },
                    "required": ["question"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "add_user_research_question",
                "description": "Add a user research question about the reflection app experience",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "The user research question to add"
                        }
                    },
                    "required": ["question"]
                }
            }
        }
    ]

    # Get initial user input
    initial_user_input = input("Please enter your initial reflection: ")
    conversation[0]["content"] = conversation[0]["content"].format(initial_user_input=initial_user_input)

    # Generate reflection questions
    while len(reflection_questions) < 3:
        response = get_openai_response(conversation, tools)
        for tool_call in response.choices[0].message.tool_calls:
            if tool_call.function.name == "add_reflection_question":
                function_args = json.loads(tool_call.function.arguments)
                add_reflection_question(function_args["question"])

    # Get user responses to reflection questions
    for i, (key, value) in enumerate(reflection_questions.items(), 1):
        print(f"\nReflection Question {i}: {value['question']}")
        answer = input("Your response: ")
        reflection_questions[key]["answer"] = answer

    # Prepare user research system prompt
    user_research_prompt = f"""You are doing user research for a reflection app.

This was the original reflection: {initial_user_input}

These were the reflection questions and answers:
reflection question 1: {reflection_questions['question_1']['question']}
reflection answer 1: {reflection_questions['question_1']['answer']}

reflection question 2: {reflection_questions['question_2']['question']}
reflection answer 2: {reflection_questions['question_2']['answer']}

reflection question 3: {reflection_questions['question_3']['question']}
reflection answer 3: {reflection_questions['question_3']['answer']}

Your task is to create another reflection question, but this one is to gather user research about their experience using the reflection app.

Focus on {user_research_focus}.

Use your "add_user_research_question" tool to ask the question.
"""

    conversation = [
        {"role": "system", "content": user_research_prompt},
    ]

    # Generate user research question
    response = get_openai_response(conversation, tools)
    user_research_question = ""
    for tool_call in response.choices[0].message.tool_calls:
        if tool_call.function.name == "add_user_research_question":
            function_args = json.loads(tool_call.function.arguments)
            user_research_question = function_args["question"]

    # Get user feedback
    print(f"\nUser Research Question: {user_research_question}")
    user_feedback = input("Your response: ")
    save_user_feedback(user_feedback)

    print("\nThank you for helping us create a better reflection experience.")

# Start the interactive chat
interactive_chat()

Please enter your initial reflection: I’m thinking about where I should move when my lease is up in September. I think that I want to be in venice as there are a lot of creative tech people but unsure if  I should go home and save money first. I have one client at home but can still work with them remote. I think there’s potential for more clients and more exciting projects in venice but rent is expensive. I have a year of savings I could use even if I don’t make any more money but  spending that is not ideal obviously.

Reflection Question 1: What are the most important factors for you in choosing a new place to live, and how do Venice and your hometown compare in those areas?
Your response: Sunny weather is nice, the people and conversations I can have and new things to learn, and well as friends. Three of my best friends are currently living in la. But also cost is important because otherwise it is not possible.

Reflection Question 2: How do you envision your personal and professio